<a href="https://colab.research.google.com/github/BoJavs-svg/LLM_Lora_FineTunning/blob/main/nb/Qwen3_(14B)-Reasoning-Conversational.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Installation

In [ ]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.15.2 triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1" huggingface_hub hf_transfer
    !pip install --no-deps unsloth
from huggingface_hub import login


### Unsloth

In [ ]:
from unsloth import FastLanguageModel
from transformers import AutoTokenizer

# 1. Load the base model
base_model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen2.5-14B",
    max_seq_length = 2048,
    load_in_4bit = True,
    load_in_8bit = False,
    full_finetuning = False,
)

# 2. Reapply LoRA the same way as originally done
model = FastLanguageModel.get_peft_model(
    base_model,
    r = 32,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 32,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
)

# 3. Load adapter weights directly
model.load_adapter("BoJavs/TrainedQwen2.5", adapter_name="default")
model.set_adapter("default")
tokenizer.add_special_tokens({
    "additional_special_tokens": ["<|im_start|>", "<|im_end|>"]
})

# Set the chat template manually (Jinja2-style template)
tokenizer.chat_template = """{% for message in messages %}
{{ '<|im_start|>' + message['role'] + '\n' + message['content'] + '<|im_end|>\n' }}
{% endfor %}
{% if add_generation_prompt %}
<|im_start|>assistant
{% endif %}"""
# 4. Continue with training or inference
print(sum(p.requires_grad for p in model.parameters()))  # Should be > 0


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.5.9: Fast Qwen2 patching. Transformers: 4.52.2.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json:   0%|          | 0.00/196k [00:00<?, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/2.12G [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/617 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

Unsloth 2025.5.9 patched 48 layers with 48 QKV layers, 48 O layers and 48 MLP layers.


adapter_model.safetensors:   0%|          | 0.00/551M [00:00<?, ?B/s]

672


We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 32,           # Choose any number > 0! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 32,  # Best to choose alpha = rank or rank*2
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
)

Unsloth: Already have LoRA adapters! We shall skip this step.


<a name="Data"></a>
### Data Prep
Qwen3 has both reasoning and a non reasoning mode. So, we should use 2 datasets:

1. We use the [Open Math Reasoning]() dataset which was used to win the [AIMO](https://www.kaggle.com/competitions/ai-mathematical-olympiad-progress-prize-2/leaderboard) (AI Mathematical Olympiad - Progress Prize 2) challenge! We sample 10% of verifiable reasoning traces that used DeepSeek R1, and whicht got > 95% accuracy.

2. We also leverage [Maxime Labonne's FineTome-100k](https://huggingface.co/datasets/mlabonne/FineTome-100k) dataset in ShareGPT style. But we need to convert it to HuggingFace's normal multiturn format as well.

In [ ]:
from datasets import load_dataset
swe_bench_lite = load_dataset('BoJavs/Clean_SweBench', split='train')

README.md:   0%|          | 0.00/861 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/739k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/172k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/176 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/44 [00:00<?, ? examples/s]

Let's see the structure of both datasets:

In [ ]:
swe_bench_lite

Dataset({
    features: ['repo', 'instance_id', 'base_commit', 'patch', 'test_patch', 'problem_statement', 'hints_text', 'created_at', 'version', 'FAIL_TO_PASS', 'PASS_TO_PASS', 'environment_setup_commit', 'image_name'],
    num_rows: 176
})

Next we take the non reasoning dataset and convert it to conversational format as well.

> Agregar bloque entrecomillado



We have to use Unsloth's `standardize_sharegpt` function to fix up the format of the dataset first.

In [ ]:
def separate_patch(patch_text):
    old_lines = []
    new_lines = []

    for line in patch_text.splitlines():
        if line.startswith('@@'):
            # diff hunk header, ignore or you can parse line ranges if needed
            continue
        elif line.startswith('-'):
            # removed line: add to old code only
            old_lines.append(line[1:])
        elif line.startswith('+'):
            # added line: add to new code only
            new_lines.append(line[1:])
        else:
            # context line, add to both old and new
            old_lines.append(line)
            new_lines.append(line)

    old_code = "\n".join(old_lines)
    new_code = "\n".join(new_lines)

    return old_code, new_code


In [ ]:
def generate_conversation(batch):
    conversations = []

    for problem, patch, repo in zip(
        batch["problem_statement"],
        batch["patch"],
        batch["repo"],
    ):
       prev, new = separate_patch(patch)
        user_prompt = f"""\
We're currently solving the following issue within our repository. Here's the issue text:
ISSUE:
{problem}
Now, you're going to solve this issue on your own.
The issue is in:
{prev}
YOU MUST RETURN A PATCH
        """
        patch= f"""\
DISCUSSION
The solved code for this problem is:
{new}
<command>
{patch}
</command>
        """
        conversations.append([
            {"role": "user", "content": user_prompt},
            {"role": "assistant", "content": patch}
        ])

    return {"conversations": conversations}


In [ ]:
from unsloth.chat_templates import standardize_sharegpt

# Standardize the dataset first
dataset = standardize_sharegpt(swe_bench_lite)

# Apply chat template with explicit thinking mode
swe_bench_lite_conversations = tokenizer.apply_chat_template(
    dataset.map(generate_conversation, batched = True)["conversations"],
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=False  # Explicitly disable thinking mode
)

Map:   0%|          | 0/176 [00:00<?, ? examples/s]

Let's see the first row

In [ ]:
swe_bench_lite_conversations[1]


'<|im_start|>user\nYou are an autonomous programmer, and you\'re working directly in the command line with a special interface.\nWe\'re currently solving the following issue within our repository. Here\'s the issue text:\nISSUE:\n"default.html" deprecation warning raised for ManagementForm\'s\nDescription\n\t\nI have a project where I never render forms with the {{ form }} expression. However, I\'m still getting the new template deprecation warning because of the formset management form production, during which the template used is insignificant (only hidden inputs are produced).\nIs it worth special-casing this and avoid producing the warning for the management forms?\n\nNow, you\'re going to solve this issue on your own.\nYou need to format your output using one field; command.\nYour output should always_one_ command field EXACTLY as in the following example:\n<command>\nls -a\n</command>\nGenerate a patch.\n        <|im_end|>\n\n<|im_start|>assistant\n        <command>\n        diff

Now let's see how long both datasets are:

In [ ]:
print(len(swe_bench_lite_conversations))

176


Finally combine both datasets:

In [ ]:
import pandas as pd
data = pd.Series(swe_bench_lite_conversations)
data.name = "text"

from datasets import Dataset
dataset = Dataset.from_pandas(pd.DataFrame(data))
dataset = dataset.shuffle(seed = 3407)

<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`.

In [ ]:
from trl import SFTTrainer, SFTConfig
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    eval_dataset = None, # Can set up evaluation!
    args = SFTConfig(
        dataset_text_field = "text",
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4, # Use GA to mimic batch size!
        # warmup_steps = 5,
        num_train_epochs = 1, # Set this for 1 full training run.
        # max_steps = 15,
        learning_rate = 2e-4, # Reduce to 2e-5 for long training runs
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        report_to = "none", # Use this for WandB etc
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=12):   0%|          | 0/176 [00:00<?, ? examples/s]

Let's train the model! To resume a training run, set `trainer.train(resume_from_checkpoint = True)`

In [ ]:

for _ in range(50):
  print(_)
  trainer_stats = trainer.train()
  model.save_pretrained("lora_model")  # Local saving
  tokenizer.save_pretrained("lora_model")
  model.push_to_hub_merged(
      "BoJavs/TrainedQwen2.5",
      tokenizer,
      save_method="lora",
  )

0


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 176 | Num Epochs = 1 | Total steps = 22
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 137,625,600/14,000,000,000 (0.98% trained)


Step,Training Loss
1,0.111700
2,0.106600
3,0.084600
4,0.101100
5,0.093700
6,0.092900
7,0.088500
8,0.133000
9,0.097800
10,0.106100


KeyboardInterrupt: 

<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [ ]:
model.save_pretrained_merged(
    "merged_model",
    tokenizer,
    save_method="merged_16bit",
)

model.save_pretrained_gguf("gguf_model", tokenizer, quantization_method="q4_k_m")
model.push_to_hub_gguf(
    "BoJavs/TrainedQwen2.5-GGUF",
    tokenizer,
    quantization_method=["q4_k_m"],
)


Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 12.1G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 44.61 out of 83.48 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


 67%|██████▋   | 32/48 [00:00<00:00, 53.35it/s]
We will save to Disk and not RAM now.
100%|██████████| 48/48 [00:20<00:00,  2.36it/s]


Unsloth: Saving tokenizer... Done.
Done.
Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 50.15 out of 83.48 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


100%|██████████| 48/48 [00:10<00:00,  4.46it/s]


...

Unsloth: Saving tokenizer... Done.
Done.
==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits might take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['q4_k_m', 'q8_0'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...
Unsloth: [1] Converting model at BoJavs/TrainedQwen2.5-GGUF into bf16 GGUF format.
The output location will be /content/BoJavs/TrainedQwen2.5-GGUF/unsloth.BF16.gguf
This might take 3 minutes...
INFO:hf-to-gguf:Loading model: TrainedQwen2.5-GGUF
INFO:hf-to-gguf:Model architecture: Qwen2ForCausalLM
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:gguf: loading model weight map from 'model.safetensors.index.json'
INFO:hf-to-gguf:gguf: loading model part 'model-00001-of-000

unsloth.Q4_K_M.gguf:   0%|          | 0.00/8.99G [00:00<?, ?B/s]

Saved GGUF to https://huggingface.co/BoJavs/TrainedQwen2.5-GGUF
Unsloth: Uploading GGUF to Huggingface Hub...


unsloth.Q8_0.gguf:   0%|          | 0.00/15.7G [00:00<?, ?B/s]

Saved GGUF to https://huggingface.co/BoJavs/TrainedQwen2.5-GGUF
